In [1]:
# Char to Word(Squad) - Done
# Conversion the whole data into reading data with relevant information - Done
# Tokenizing the context for data extraction - Done
# Preprocessing the data- Word2vec and Char Embedding
# Making neccesary list

In [2]:
import nltk
import re
import json

In [3]:
with open('data/squad/train-v1.1.json') as json_data:
    train_data = json.load(json_data)

In [4]:
def process_context(context):
    tokens = []
    for token in context:
        flag = False
        l = ("-", "\u2212", "\u2014", "\u2013", "/", "~", '"', "'", "\u201C", "\u2019", "\u201D", "\u2018", "\u00B0")
        tokens.extend(re.split("([{}])".format("".join(l)), token))
    return tokens

def word_tokenize(tokens):
    return [token.replace("''", '"').replace("``", '"') for token in nltk.word_tokenize(tokens)]

sent_tokenize = nltk.sent_tokenize

In [5]:
def get_2d_spans(context, each_word):
    span = []
    start_index = 0
    for tokens in each_word:
        spans = []
        for token in tokens:
            start_index = context.find(token, start_index) # getting start index of each word after a particular index
            spans.append((start_index, start_index + len(token))) # appending that start and end index into list
            start_index += len(token) # updating the start index to check ahead
        span.append(spans)
    return span

def get_word_span(context, each_word, start, stop):
    each_word_span = get_2d_spans(context, each_word)
    word_span_list = []
    for sent_index, word_span in enumerate(each_word_span):
        for word_index, char_span in enumerate(word_span):
             if (stop >= char_span[0] and start <= char_span[1]):
                word_span_list.append((sent_index, word_index))
    return word_span_list[0], (word_span_list[-1][0], word_span_list[-1][1] + 1)

In [6]:
def read_data():
    data = []
    data_points = {}
    each_context = []
    main_data = []
    for data_index, data_paragraph in enumerate(train_data['data']):
        for each_paragraph in data_paragraph['paragraphs']:
            question_data = each_paragraph['qas']
            context_data = each_paragraph['context']
            for number_of_question in range(len(question_data)):
                    data_points['context'] = context_data
                    data_points['question'] = question_data[number_of_question]['question']
                    answer = question_data[number_of_question]['answers'][0]['text']
                    data_points['answers'] = answer
                    start = question_data[number_of_question]['answers'][0]['answer_start']
                    stop = start + len(answer) 
                    tokenized_context = list(map(word_tokenize, sent_tokenize(context_data)))
                    tokenized_context = [process_context(tokens) for tokens in tokenized_context]
                    y0, y1 = get_word_span(data_points['context'], tokenized_context, start, stop)
                    data_points['answer_start'] = y0
                    data_points['answer_stop'] = y1
                    data.append(data_points)
                    data_points = {}
            each_context.append(data)
            data = []
        main_data.append(each_context)
        each_context = []
    return main_data

In [7]:
training_data = read_data()